# Imports

In [1]:
# Add `src` directory to path (change this to your `src` directory)
import sys
sys.path.insert(0, "/home/cody/abcnn/ABCNN/src")

In [3]:
# Import `standard` modules
import os
from torch.utils.data import TensorDataset

# Import custom modules
from setup import read_config
from setup import setup
from trainer.factories import loss_fn_factory
from trainer.factories import optimizer_factory
from trainer.factories import scheduler_factory
from trainer.multiclass_classifier_trainer import MulticlassClassifierTrainer
from trainer.utils import move_to_device
from utils import abcnn_model_loader

# Setup

In [4]:
# Set the input values
CONFIG_PATH = "/home/cody/abcnn/ABCNN/src/config.yaml"# path to configuration file
TRAINSET = "moveworks_train"# name of training set (should be a key in data_paths)
VALSET = "moveworks_val"# name of validation set (should be a key in data_paths)
TESTSET = "moveworks_test"# name of test set (should be a key in data_paths)
LOAD_PATH = None # load a model from a checkpoint file

In [5]:
# Sanity check input values
assert(os.path.isfile(CONFIG_PATH))
assert(LOAD_PATH is None or os.path.isfile(LOAD_PATH))

In [6]:
# Setup modules
config = read_config(CONFIG_PATH)
features, labels, model = setup(config["model"])
model = move_to_device(config["trainer"]["device"], model) # hacky, but necessary for trainer
datasets = {
    name: TensorDataset(features[name], labels[name])
    for name in features
}
loss_fn = loss_fn_factory(config["loss_fn"])
optimizer = optimizer_factory(config["optimizer"], model.parameters())
scheduler = scheduler_factory(config["scheduler"], optimizer)
trainer = MulticlassClassifierTrainer(config["trainer"])
if LOAD_PATH:
    model, optimizer = abcnn_model_loader(LOAD, model, optimizer)

moveworks_test: 100%|██████████| 437/437 [00:00<00:00, 2752.92it/s]


Loading FastText word vectors from: /home/cody/abcnn/embeddings/fasttext/tickets/word_vector_from_tickets_skipgram_dim300_subword_min2_max6.bin


embedding matrix: 100%|██████████| 2666/2666 [00:00<00:00, 128035.89it/s]


Creating the ABCNN model...


# Train the model

In [ ]:
trainset = datasets[TRAINSET]
valset = datasets[VALSET] if VALSET else None
trainer.train(loss_fn, model, optimizer, trainset, scheduler=scheduler, valset=valset)

train:   0%|          | 0/123 [00:00<?, ?it/s]/home/cody/.virtualenvs/abcnn/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")

train: 100%|██████████| 123/123 [00:32<00:00,  3.82it/s]

train:   0%|          | 0/123 [00:00<?, ?it/s]


train: 100%|██████████| 123/123 [00:32<00:00,  3.80it/s]

train:   5%|▍         | 6/123 [00:01<00:34,  3.36it/s]


train: 100%|██████████| 123/123 [00:32<00:00,  3.77it/s]

train:  10%|▉         | 12/123 [00:03<00:31,  3.48it/s]


train: 100%|██████████| 123/123 [00:32<00:00,  3.76it/s]

train:  15%|█▍        | 18/123 [00:04<00:29,  3.61it/s]


train: 100%|██████████| 123/123 [00:32<00:00,  3.78it/s]

train:  20%|█▉        | 24/123 [00:06<00:27,  3.63it/s]


train:  67%|██████▋   | 83/123 [00:22<00:10,  3.75it/s]

# Make predictions

In [ ]:
testset = datasets[TESTSET]
trainer.predict(testset)